Use oof preds to make a dataset where the model must predict at spacy token level

In [1]:
import json
import os
from dotenv import load_dotenv
from pathlib import Path

dotenv_path = Path("../../.env")
if dotenv_path.exists():
    print("Loaded .env file!")
    load_dotenv(str(dotenv_path))


data = json.load(open(Path(os.environ["PROJECT_HOME_DIR"]) / "data/corrected_train_v1.json"))

Loaded .env file!


In [2]:
import pandas as pd

pd.options.display.max_rows = 999

df = pd.concat([
    pd.read_parquet("/drive2/kaggle/pii-dd/piidd/inference/outputs/d3l-romulan-seven-836/preds.pq"),
    pd.read_parquet("/drive2/kaggle/pii-dd/piidd/inference/outputs/d3l-defiant-picard-837/preds.pq"),
    pd.read_parquet("/drive2/kaggle/pii-dd/piidd/inference/outputs/d3l-tough-plasma-838/preds.pq"),
    pd.read_parquet("/drive2/kaggle/pii-dd/piidd/inference/outputs/d3l-exalted-dream-839/preds.pq"),
])

cdf = pd.read_parquet("/drive2/kaggle/pii-dd/data/corrected_train_v1.pq")

df = df.merge(cdf[["token", "document", "label"]], on=["token", "document"], how="left")

df

,token,document,token_text,B-EMAIL,B-ID_NUM,B-NAME_STUDENT,B-PHONE_NUM,B-STREET_ADDRESS,B-URL_PERSONAL,B-USERNAME,I-ID_NUM,I-NAME_STUDENT,I-PHONE_NUM,I-STREET_ADDRESS,O,row_id,label
0,0,16,Reporting,6.613203e-07,6.375219e-07,3.897991e-06,3.407962e-07,5.004425e-07,7.379113e-06,1.843209e-06,4.996189e-07,9.175893e-07,8.876922e-07,5.153819e-06,0.999977,0,O
1,1,16,process,1.917580e-07,3.647256e-07,1.977124e-06,1.240302e-07,4.011568e-07,1.791013e-06,8.325435e-07,2.059751e-07,6.049944e-07,2.593306e-07,1.012589e-06,0.999992,1,O
2,2,16,\n\n,2.235166e-07,4.102733e-07,1.879485e-06,2.749369e-07,5.233018e-07,2.927439e-06,6.118441e-07,2.625444e-07,5.053022e-07,4.464375e-07,1.500160e-06,0.999991,2,O
3,3,16,by,4.035668e-07,7.022506e-07,4.340441e-06,4.510157e-07,7.352551e-07,3.465557e-06,5.955202e-07,4.082351e-07,1.068870e-06,3.314971e-07,1.463319e-06,0.999986,3,O
4,4,16,Gilberto,6.296745e-05,1.039265e-04,9.928824e-01,3.584145e-05,4.770503e-05,2.085956e-04,4.689154e-05,3.979826e-05,1.658336e-04,4.272398e-05,4.652205e-05,0.006317,4,B-NAME_STUDENT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4764472,814,22687,the,7.952080e-08,8.473634e-08,1.958464e-07,4.488912e-08,8.632806e-08,2.008846e-07,1.096388e-07,6.479080e-08,1.001370e-07,5.318639e-08,8.583386e-08,0.999999,1173511,O
4764473,815,22687,process,6.918367e-08,1.005586e-07,2.296683e-07,6.003002e-08,9.353393e-08,1.848233e-07,1.274801e-07,8.898299e-08,9.266525e-08,5.531047e-08,8.954944e-08,0.999999,1173512,O
4764474,816,22687,explained,7.209870e-08,1.453342e-07,1.723834e-07,6.014898e-08,1.005618e-07,2.518434e-07,1.185082e-07,1.048142e-07,1.425418e-07,7.976565e-08,9.615538e-08,0.999999,1173513,O
4764475,817,22687,above,5.971995e-08,9.539418e-08,3.022207e-07,5.186531e-08,1.124224e-07,2.014887e-07,1.286466e-07,9.235575e-08,1.505900e-07,5.453694e-08,7.164208e-08,0.999999,1173514,O


In [3]:
small_df = df[(
    df[['B-EMAIL', 'B-ID_NUM',
       'B-NAME_STUDENT', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'B-URL_PERSONAL',
       'B-USERNAME', 'I-ID_NUM', 'I-NAME_STUDENT', 'I-PHONE_NUM',
       'I-STREET_ADDRESS',]].sum(axis=1) > 0.05) |
       (df["O"] < 0.99) |
       (df["label"] != "O")
       ].copy().reset_index(drop=True)

small_df

,token,document,token_text,B-EMAIL,B-ID_NUM,B-NAME_STUDENT,B-PHONE_NUM,B-STREET_ADDRESS,B-URL_PERSONAL,B-USERNAME,I-ID_NUM,I-NAME_STUDENT,I-PHONE_NUM,I-STREET_ADDRESS,O,row_id,label
0,4,16,Gilberto,0.000063,0.000104,0.992882,0.000036,0.000048,0.000209,0.000047,0.000040,0.000166,0.000043,0.000047,0.006317,4,B-NAME_STUDENT
1,5,16,Gamboa,0.000204,0.000151,0.001485,0.000038,0.000108,0.000383,0.000079,0.000147,0.981160,0.000148,0.000296,0.025361,5,I-NAME_STUDENT
2,5,20,Sindy,0.000057,0.000151,0.987762,0.000018,0.000114,0.000217,0.000045,0.000057,0.000106,0.000028,0.000053,0.011594,733,B-NAME_STUDENT
3,6,20,Samaca,0.000261,0.000451,0.004944,0.000047,0.000167,0.000815,0.000234,0.000251,0.935954,0.000238,0.000995,0.061239,734,I-NAME_STUDENT
4,8,20,Gitam,0.000695,0.000817,0.075716,0.000217,0.004622,0.004030,0.001404,0.000503,0.003982,0.000676,0.004898,0.926504,736,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65027,34,22663,,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1171512,O
65028,43,22663,,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1171521,O
65029,87,22663,,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1171562,O
65030,638,22663,,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1172086,O


In [4]:
wrong_names = small_df[(small_df["B-NAME_STUDENT"] > 0.2) & (small_df["label"] == "O")]
len(wrong_names)

574

In [5]:
wrong_names

,token,document,token_text,B-EMAIL,B-ID_NUM,B-NAME_STUDENT,B-PHONE_NUM,B-STREET_ADDRESS,B-URL_PERSONAL,B-USERNAME,I-ID_NUM,I-NAME_STUDENT,I-PHONE_NUM,I-STREET_ADDRESS,O,row_id,label
6,328,20,George,0.000037,0.000073,0.988454,0.000042,0.000052,0.000114,0.000061,0.000056,0.000216,0.000056,0.000032,0.013443,1055,O
7,330,20,Geoff,0.000067,0.000165,0.978476,0.000053,0.000077,0.000210,0.000079,0.000106,0.000599,0.000114,0.000068,0.022712,1057,O
930,7,104,Dr,0.000732,0.001191,0.710423,0.000509,0.002104,0.000834,0.000532,0.000104,0.003849,0.000500,0.000539,0.278683,3729,O
972,1530,308,MCMR,0.000390,0.001188,0.202706,0.000160,0.000839,0.001179,0.000522,0.000280,0.000906,0.000429,0.000376,0.959372,8742,O
973,1532,308,Hunt,0.000363,0.000373,0.766674,0.000185,0.000367,0.001091,0.000607,0.000595,0.004894,0.000384,0.000772,0.293110,8744,O
974,1542,308,Garda,0.000135,0.000250,0.801489,0.000117,0.000314,0.001058,0.000609,0.000324,0.002621,0.000155,0.000661,0.244379,8754,O
977,1549,308,Hunt,0.000348,0.000348,0.749123,0.000229,0.000439,0.000954,0.000543,0.000436,0.004852,0.000338,0.000579,0.304455,8761,O
978,1559,308,Garda,0.000113,0.000204,0.857223,0.000123,0.000302,0.000779,0.000675,0.000245,0.002021,0.000115,0.000529,0.206993,8771,O
1123,298,3540,Fabian,0.000489,0.001172,0.574207,0.000381,0.001114,0.025094,0.000534,0.000684,0.002086,0.000500,0.000660,0.553895,24295,O
1205,251,4236,Manizales,0.000451,0.000470,0.294714,0.000453,0.001415,0.003511,0.000406,0.000497,0.007861,0.000495,0.000878,0.832060,33651,O


In [6]:
small_df[["token", "document", "token_text", "label"]].to_parquet("oof_ds.pq")

In [18]:
mixtral_data = json.load(open(Path(os.environ["PROJECT_HOME_DIR"]) / "data/mixtral-v3.json"))

In [20]:
mixtral_rows = []
for m in mixtral_data:

    for i, (token, label) in enumerate(zip(m["tokens"], m["labels"])):
        if label != "O":
            mixtral_rows.append({
                "token": i,
                "document": m["document"],
                "token_text": token,
                "label": label
            })

In [21]:
mdf = pd.DataFrame(mixtral_rows, columns=["token", "document", "token_text", "label"])
mdf

,token,document,token_text,label
0,0,0,Erastus,B-NAME_STUDENT
1,1,0,Whelan,I-NAME_STUDENT
2,5,0,+4428,B-PHONE_NUM
3,6,0,9018827,I-PHONE_NUM
4,10,0,nickles,B-USERNAME
...,...,...,...,...
104771,954,2664,Yolaine,B-NAME_STUDENT
104772,956,2664,Knudsen,B-NAME_STUDENT
104773,1105,2664,https://www.t_tompkins.me/share-resource/produ...,B-URL_PERSONAL
104774,1112,2664,Saresa,B-NAME_STUDENT


In [25]:
mdf["len"] = mdf["token_text"].apply(len)

In [28]:
mdf[mdf.len == 1].label.value_counts()

I-STREET_ADDRESS    11710
I-PHONE_NUM          3520
I-NAME_STUDENT       1277
I-ID_NUM              916
B-PHONE_NUM           908
B-STREET_ADDRESS      188
B-ID_NUM                5
Name: label, dtype: int64

In [30]:
mdf[(mdf.len == 1) & (mdf.label == "I-NAME_STUDENT")].token_text.value_counts()

-    1277
Name: token_text, dtype: int64

In [24]:
mdf.head(500)

,token,document,token_text,label
0,0,0,Erastus,B-NAME_STUDENT
1,1,0,Whelan,I-NAME_STUDENT
2,5,0,+4428,B-PHONE_NUM
3,6,0,9018827,I-PHONE_NUM
4,10,0,nickles,B-USERNAME
5,12,0,collado9@rutgers.edu,B-EMAIL
6,16,0,https://www.linkedin.com/ameerahw,B-URL_PERSONAL
7,18,0,52,B-STREET_ADDRESS
8,19,0,Danny,I-STREET_ADDRESS
9,20,0,highway,I-STREET_ADDRESS


In [32]:
mdf.to_parquet("mixtral-v3.pq")